<a href="https://colab.research.google.com/github/jykim9280/KISTI_AI/blob/jiyeon/BERT_%EC%A7%80%EC%97%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

In [7]:
import os
import json
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

# Google Drive에서 데이터 파일 경로 설정
folder_path = '/content/drive/MyDrive/KISTI_AI/국정감사/16'  # Google Drive의 실제 경로를 사용

# 학습 데이터 저장할 리스트
training_data = []

# 폴더 내 모든 JSON 파일을 불러오기
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)

        try:
            # JSON 파일 열기
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

                # 질문과 답변 추출 (데이터 구조에 따라 수정 필요)
                question = data.get('question', {}).get('comment', '')
                answer = data.get('answer', {}).get('comment', '')
                context = data.get('context', '')

                if question and answer:
                    # 학습 데이터 리스트에 추가
                    training_data.append({"question": question, "answer": answer, "context": context})
        except Exception as e:
            print(f"Error loading {filename}: {e}")

# 학습용 텍스트 파일로 저장
with open('training_data.json', 'w', encoding='utf-8') as f:
    json.dump(training_data, f, ensure_ascii=False, indent=4)


In [8]:
# 데이터셋 로드
dataset = load_dataset('json', data_files={'train': 'training_data.json'})

# Hugging Face BERT 모델 및 토크나이저 불러오기
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a B

In [9]:
# GPU/CPU 설정 (Colab에서는 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # 모델을 GPU로 이동

# 데이터셋 토큰화 (질문, 문맥 및 답변 위치)
def tokenize_function(examples):
    questions = examples['question']
    contexts = examples['context']
    answers = examples['answer']

    inputs = tokenizer(questions, contexts, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

    # start_positions와 end_positions 계산
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        answer = answers[i]
        context = contexts[i]

        # 답변이 문맥에서 어디에 있는지 찾기
        start_idx = context.find(answer)
        if start_idx == -1:
            start_idx = 0  # 답변을 찾지 못하면 0으로 설정
        end_idx = start_idx + len(answer)

        start_positions.append(start_idx)
        end_positions.append(end_idx)

    inputs['start_positions'] = torch.tensor(start_positions).to(device)
    inputs['end_positions'] = torch.tensor(end_positions).to(device)

    return inputs

# 데이터셋에 대해 토큰화 적용
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [11]:
# 손실 함수 정의 및 Trainer 설정
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        start_positions = inputs.pop("start_positions")
        end_positions = inputs.pop("end_positions")
        outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,         # GPU에서는 더 큰 배치 크기 사용 가능
    gradient_accumulation_steps=4,         # 그래디언트 누적
    num_train_epochs=3,                    # 학습 에포크 수
    logging_dir='./logs',                  # 로그 저장 경로
)

# Trainer 초기화
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)

# 학습 시작
trainer.train()

# 모델 저장
model.save_pretrained("./fine_tuned_bert")
tokenizer.save_pretrained("./fine_tuned_bert")

# 나중에 모델 불러오기
model = BertForQuestionAnswering.from_pretrained("./fine_tuned_bert")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_bert")


Step,Training Loss


In [31]:
# BERT 모델을 이용한 질의응답 처리 함수
def answer_question(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")

    print(f"Question: {question}")
    print(f"Context: {context}")  # 문맥이 제대로 입력되었는지 확인

    with torch.no_grad():
        outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer


In [32]:
import os
import json
import re

# 간단한 키워드 추출 함수 (명사 기반 추출, Mecab 등 사용 가능)
def extract_keywords(text):
    # 간단히 정규식을 사용한 명사 추출 (Mecab 등 전문 도구 사용 권장)
    keywords = re.findall(r'\b\w+\b', text)
    return keywords

# 검증 함수 (키워드 기반으로 문맥 찾기)
def evaluate_model():
    folder_path = '/content/drive/MyDrive/KISTI_AI/국정감사/16'  # JSON 데이터 경로

    while True:
        test_question = input("질문을 입력하세요 (종료하려면 '종료'라고 입력하세요): ")

        if test_question.lower() == "종료":
            print("챗봇을 종료합니다.")
            break

        # 키워드 추출 (Mecab 같은 형태소 분석기를 사용하는 것이 더 정확함)
        question_keywords = extract_keywords(test_question)

        found_context = ""
        for filename in os.listdir(folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(folder_path, filename)

                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    context = data.get('context', '')
                    question = data.get('question', {}).get('comment', '')

                    # JSON 파일의 질문에서도 키워드를 추출
                    question_in_file_keywords = extract_keywords(question)

                    # 키워드가 겹치는지 확인 (키워드가 겹치면 문맥을 찾은 것으로 간주)
                    if any(keyword in question_in_file_keywords for keyword in question_keywords):
                        found_context = context
                        break

        if not found_context:
            print("문맥을 찾지 못했습니다. 데이터에서 관련 문맥을 찾을 수 없습니다.")
            continue

        print(f"\n[질문]: {test_question}")
        print(f"[문맥]: {found_context}")

        # 문맥을 모델에 전달하여 답변 생성
        response = answer_question(test_question, found_context)

        print("\n[대답]:", response)
        print("\n")

if __name__ == "__main__":
    evaluate_model()


질문을 입력하세요 (종료하려면 '종료'라고 입력하세요): 건설교통부 산하 12개 기관의 부채가 얼마야?
문맥을 찾지 못했습니다. 데이터에서 관련 문맥을 찾을 수 없습니다.
질문을 입력하세요 (종료하려면 '종료'라고 입력하세요): 건설교통부 산하 12개 기관의 부채
문맥을 찾지 못했습니다. 데이터에서 관련 문맥을 찾을 수 없습니다.
질문을 입력하세요 (종료하려면 '종료'라고 입력하세요): SOFA

[질문]: SOFA
[문맥]: 그 답변 이전에 SOFA 관련해서 제가 물어 보았던 문제의 핵심은 미국 쪽에서도 물론 환경문제를 무시하는 것 같지는 않습니다.  그런데 환경문제를 SOFA의 본문에 별도의 환경조항으로 신설하려고 하지 않고 양해각서나 아니면 부속문서 그런 식으로 하는 것이 아닌가?  그런 여러 가지 관련된 이야기가 나오고 있기 때문에 제가 그 질의를 했는데 장관께서 그 부분에 대해서 명확하게 답변을 안 하시고 넘어가시는 것 같은데 미국 쪽의 입장이 어떻습니까?  환경을 SOFA 본문조항에 포함시키는 대신에 양해각서나 다른 부속문서를 통해 가지고 실질적인 환경보호 그런 부분들로 대체하자는 것 아닌가요? 이와 관련해서는 최근에 미국과 일본 사이에 별도의 문서로서 이 문제를 규율하고 있다.  여러 위원님들이 그것을 염두에 두고 여러 가지 말씀을 하신 것으로 제가 이해를 합니다.   문제는 SOFA 협정에 어떤 내용의 환경조항이 들어가느냐, 그 내용의 문제가 되겠습니다.  그래서 그것은 저희들도 그동안에 저희의 입장을 충분히 개진했고 미측에서도 방금 金 위원님 말씀하신 대로 저희 입장을 충분히 이해하고 있기 때문에 이번 11월 회의에서 초안 제시할 때 그때 아마 구체적으로 파악이 되지 않을까 생각됩니다.  조금만 기다려 주시고 11월의 협상결과는 별도로 저희들이 보고를 드리겠습니다. 이 점에 대해서 제가 한 가지만 여쭙겠습니다.   지금 SOFA의 틀 내에서 환경문제를 다루는 것을 정부에서 골자로 하고 있습니까, 그렇지 않으면 한국 환경법에 주한미군의 여러 가지

RuntimeError: The size of tensor a (1674) must match the size of tensor b (512) at non-singleton dimension 1

In [15]:
# # JSON 파일에서 question, answer, context를 정확히 추출했는지 확인
# for filename in os.listdir(folder_path):
#     if filename.endswith(".json"):
#         file_path = os.path.join(folder_path, filename)

#         try:
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 data = json.load(f)

#                 question = data.get('question', {}).get('comment', '')
#                 answer = data.get('answer', {}).get('comment', '')
#                 context = data.get('context', '')  # 문맥을 정확히 추출

#                 print(f"Question: {question}")
#                 print(f"Context: {context}")
#                 print(f"Answer: {answer}")

#         except Exception as e:
#             print(f"Error loading {filename}: {e}")


Question: 정품이 단종이 되었을 경우에는 대체품을 쓸 수밖에 없지 않습니까?  그런데 그 제품의 물리적인 특성과 또 설치된 계통의 기능이 일치하는지 또 정품을 쓰지 않았을 때 문제점은 없는지에 대해서 말씀해 주십시오.
Context: 수고 많으십니다.    9월 4일이니까 지금부터 열흘 전에 옆에 계시는 金榮春 위원이 주관되어가지고 월성원전부지 안전문제에 대해서 국회에서 공개토론회를 했습니다.  알고 계십니까? 알고 있습니다. 신문에도 보도가 되었는데 그날 학계 전문가들 학자들, 환경전문가들도 많이 와서 아주 의미깊은 토론회를 하루종일 했는데 그 내용을 보신 적이 있습니까? 예, 본사를 통해서 받아보았습니다. 그래서 저는 월성원전 부지문제에 대해서 너무 지나치게 위험성이 있는 것처럼 과대포장해서 국민을 불안하게 해서도 안 된다고 생각하고 있고 또 그러나 많은 전문가들이 어쨌든 역사적으로 지진이 많이 일어나고 만에 하나라도 원전에 무슨 사고가 있다면 그것은 이 지역 뿐만 아니라 이 나라, 심지어 세계적인 재앙이 될 수 있기 때문에 절대 가볍게 봐서는 안 된다고 봅니다.  그래서 그 문제에 대해서 지금 신월성 1, 2호기를 건설중인데 그것을 바로 건설해야 된다 아니면 나중에 결과를 보고 해야 된다 하는 여러 가지 의견들이 많은데 그에 대한 의견을 말씀해 주시기 바랍니다.  그 토론회를 보고 느낀 점 또 그에 대해서 그렇기 때문에 어떻게 했으면 좋겠다는 인식을 간단히 말씀해 주세요. 우선 제가 지질 내지 지진전문가가 아니라는 점을 우선 말씀을 드리겠습니다.  그래서 위원님 질의의 핵심에 벗어나는 답변이 될지 걱정됩니다마는 金 위원님이 주관하셔서 개최한 회의기록을 봤습니다.  상당히 의견을 달리하는 학자분들을 모셔다 놓고 하셨는데 저는 깜짝 놀랐습니다.  이렇게 의정활동을 하시는데 참고를 하시려고 이런 활동을 하는 것에 대해서 놀랐고 저는 기술적으로 전문가는 아닙니다마는 양쪽 의견 외에도 또 중재의견이라면 어패가 있겠습니다마는 상당히 균형잡힌 논쟁이 계속 진행